# Extracting stock prices of individual companies

In [1]:
import datetime as dt
import pandas as pd
import pandas_datareader.data as web
start=dt.datetime(2018,11,16)
end=dt.datetime(2018,11,30)

In [2]:
dates = ['2018-11-16','2018-11-19','2018-11-20','2018-11-21','2018-11-23','2018-11-26','2018-11-27','2018-11-28','2018-11-29','2018-11-30']

In [3]:
company_open = []
company_close = []
volume = []
dow_jones = pd.read_csv('dow_jones.csv')
ticker = dow_jones['Ticker'].tolist()
for company in ticker:
    df=web.DataReader(company,'yahoo',start,end)    #API call to extract stock price data of a company
    company_open.append(df['Open'].tolist())        #to extract opening stock price
    company_close.append(df['Close'].tolist())      #to extract closing stock price
    volume.append(df['Volume'].tolist())            #to extract volume of stocks traded   

In [4]:
dow_jones = pd.read_csv('dow_jones.csv')
ticker = dow_jones['Ticker'].tolist()

In [5]:
previous_1 = []
previous_2 = []
previous_3 = []
previous_4 = []
previous_5 = []
company = []
date = []
volumetraded = []
company_percent = []
for i in range(0,len(ticker)):
    for j in range(5,10):    #to loop from Nov 26 to Nov 30
        company.append('$'+ticker[i])
        date.append(dates[j])
        company_percent.append(((company_close[i][j]-company_open[i][j])/company_open[i][j])*100)   #percentage change in stock price
        previous_1.append(((company_close[i][j-1]-company_open[i][j-1])/company_open[i][j-1])*100)  #previous day's percentage change
        previous_2.append(((company_close[i][j-2]-company_open[i][j-2])/company_open[i][j-2])*100)
        previous_3.append(((company_close[i][j-3]-company_open[i][j-3])/company_open[i][j-3])*100)
        previous_4.append(((company_close[i][j-4]-company_open[i][j-4])/company_open[i][j-4])*100)
        previous_5.append(((company_close[i][j-5]-company_open[i][j-5])/company_open[i][j-5])*100)
        volumetraded.append(volume[i][j-1])    #previous day's volume traded

In [6]:
percentchanges = pd.DataFrame({'Company':company,'Date':date,'PercentChange':company_percent,
                               'previous_1':previous_1,'previous_2':previous_2,
                              'previous_3':previous_3,'previous_4':previous_4,'previous_5':previous_5})
percentchanges.to_csv('percentchanges.csv', index=False)

# Extracting DJIA values

In [7]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options = chrome_options)

In [8]:
import datetime
import time

today = datetime.datetime.today().strftime('%b %d, %Y')
unix_today = time.mktime(datetime.datetime.strptime(today,'%b %d, %Y').timetuple())
#print(unix_today)

In [9]:
#url = 'https://finance.yahoo.com/quote/%5EDJI/history?period1=1512104400&period2=' + str(unix_today) + '&interval=1d&filter=history&frequency=1d'
url = "https://finance.yahoo.com/quote/%5EDJI/history?period1=1542344400&period2=1543554000&interval=1d&filter=history&frequency=1d"

driver.get(url)
assert '^DJI' in driver.title # Wait for the page to load #could even put in a sleep command to allow the code to stay still a bit before moving to the next line
html = driver.page_source # Get the html of the page
soup = bs(html, 'html5lib')

details_div = ""
for div in soup.find_all('div'):
    if(div.get('class') is not None):
        if(len(div.get('class'))>0 and ' '.join(div.get('class')) == "Pb(10px) Ovx(a) W(100%)"):
            details_div = div
            
tr_list = []  
#print(details_div)
for tr in details_div.find_all('tr'):
    if(tr.get('class') is not None):
        if(' '.join(tr.get('class')) == "BdT Bdc($c-fuji-grey-c) Ta(end) Fz(s) Whs(nw)"):
            tr_list.append(tr)

driver.quit

<bound method WebDriver.quit of <selenium.webdriver.chrome.webdriver.WebDriver (session="4b296e9a1e82389c1a88c02da038f548")>>

In [10]:
details_list = []
for tr in tr_list[0:10]:
    current_td_list = []
    current_details = {}
    for td in tr.find_all('td'):
        current_td_list.append(td.find('span').text)
    current_details['date'] = current_td_list[0]
    current_details['open'] = current_td_list[1]    
    current_details['high'] = current_td_list[2]    
    current_details['low'] = current_td_list[3]    
    current_details['close'] = current_td_list[4]    
    current_details['adjclose'] = current_td_list[5]    
    current_details['vol'] = current_td_list[6]    
    details_list.append(current_td_list)
    

In [11]:
date = []
open_price = []
high_price = []
low_price = []
close_price = []
sharestraded_volume = []

for x in range (0, len(details_list)):
    date.append(details_list[x][0])
    open_price.append(details_list[x][1])
    high_price.append(details_list[x][2])
    low_price.append(details_list[x][3])
    close_price.append(details_list[x][4])
    sharestraded_volume.append(details_list[x][6])

In [12]:
import csv #use library to create csv file
with open('DJIA.csv', mode='w', newline='') as DJIA:
    DJIA_writer = csv.writer(DJIA) #helps avoid separation on commas appearing within the review text

    DJIA_writer.writerow(['Date','Open','High','Low','Close','VolumeTraded'])
    for x in range(0, len(date)):
        DJIA_writer.writerow([date[x], open_price[x], high_price[x],low_price[x],close_price[x],sharestraded_volume[x]])

# Appending Sentiment Scores and DJIA values to csv

In [13]:
from datetime import datetime,timedelta
import pandas as pd
import math

In [14]:
percentchanges = pd.read_csv('percentchanges.csv')
tweets = pd.read_csv('TweetScore.csv')
tweets['Date'] = pd.to_datetime(tweets['Date']).dt.date
percentchanges['Date'] = pd.to_datetime(percentchanges['Date'])
comments = pd.read_csv('CommentScore.csv')
comments['Date'] = pd.to_datetime(comments['Date']).dt.date
DJIA = pd.read_csv('DJIA.csv')

In [15]:
tweetavg = []
commentavg = []
dowjoneschange = []
company = percentchanges['Company'].tolist()
date = percentchanges['Date'].tolist()
for i in range(0,len(company)):                 #calculating mean sentiment scores of all companies for all days
    testdata = tweets[((tweets['Date'] == date[i].date()-timedelta(days=1)) 
                     & (tweets['Company'] == company[i]))]
    tweetavg.append((testdata['GeneralScore'].mean()+testdata['LMScore'].mean())/2)
    testdata2 = comments[((comments['Date'] == date[i].date()-timedelta(days=1)) 
                     & (comments['Company'] == company[i]))]
    commentavg.append((testdata2['GeneralScore'].mean()+testdata2['LMScore'].mean())/2)
        
for i in range(0,len(tweetavg)):
    if (math.isnan(float(tweetavg[i])) == True):
        tweetavg[i] = 0

for i in range(0,len(commentavg)):
    if (math.isnan(float(commentavg[i])) == True):
        commentavg[i] = 0

average = pd.DataFrame({'AverageTweet':tweetavg,'AverageComment':commentavg})
percentchanges = percentchanges.join(average)
percentchanges.to_csv('percentchanges.csv', index=False)

In [16]:
requireddates = ['']        
change = DJIA['Date'].tolist()
opendow = DJIA['Open'].tolist()
closedow = DJIA['Close'].tolist()
lowdow = DJIA['Low'].tolist()
highdow = DJIA['High'].tolist()
volume = DJIA['VolumeTraded'].tolist()
dowchanges = []
DJHigh = []
DJLow = []
VolumeTraded = []
for i in range(0,len(change)):
    if (change[i] == 'Nov 26, 2018'):
        index = i
for i in range(0,30):                     #appending DJIA values
    for j in range(index,index-5,-1):
        closed = float(closedow[j+1].replace(',',''))
        opened = float(opendow[j+1].replace(',',''))
        dowchanges.append((closed-opened)/opened)
        low = float(lowdow[j+1].replace(',',''))
        high = float(highdow[j+1].replace(',',''))
        volumetrade = int(volume[j+1].replace(',',''))
        DJHigh.append(high)
        DJLow.append(low)
        VolumeTraded.append(volumetrade)
DJ = pd.DataFrame({'DJChange':dowchanges,'DJHigh':DJHigh,'DJLow':DJLow,'VolumeTraded':VolumeTraded})
percentchanges = percentchanges.join(DJ)
percentchanges.to_csv('percentchanges.csv', index=False)        